# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import pandas as pd
from sqlalchemy import create_engine

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

import pickle
import os

In [ ]:
# load data from database
engine = create_engine('sqlite:///..\\data\\DisasterMessages.db') # sqlite:/// is necessary, followed by path where the DB exists. in this case, go back one step and then drill down to the file
df = pd.read_sql_table('Messages', con=engine)
X = df['message'].values
y = df.drop(['id', 'message', 'original', 'genre'], axis=1)
category_names = y.columns.values
y = y.values

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text, lang="english"):
    
    # Normalize (Transform to lowercase and remove any punctuation)
    text = text.lower()
    pattern = re.compile('[^a-z0-9]+')
    text = pattern.sub(' ',text)
    
    # Tokenize (put text to list)
    words = word_tokenize(text)
    
    # Remove Stop Words (e.g. 'This', 'the', ...)
    words = [word for word in words if word not in stopwords.words(lang)]
    
    # Stem / Lemmatize (e.g. 'tokenizing' -> 'token')
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    words = [lemmatizer.lemmatize(lemmatizer.lemmatize(word), pos='v') for word in words]
    
    return words

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('scaler', StandardScaler(with_mean=False)),
    ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
for i, cat in enumerate(category_names):
    metrics =  classification_report(y_test[i], y_pred[i])
    print("""Category: {}
          {}
          ----------------------------------------------------
          """.format(cat, metrics))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
    'clf__estimator__n_estimators': [50, 100],
    'clf__estimator__criterion': ['entropy', 'gini'],
    'vect__ngram_range': [(1,1), (1,2)],
    'tfidf__sublinear_tf': [False, True]
}

cv = GridSearchCV(pipeline, parameters, n_jobs=-1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
y_pred_tuned = cv.predict(X_test)

In [ ]:
for i, cat in enumerate(category_names):
    metrics =  classification_report(y_test[i], y_pred_tuned[i])
    print("""Category: {}
          {}
          ----------------------------------------------------
          """.format(cat, metrics))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# Calculates the length of a single message
class MessageLengthTransformer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return [len(x) for x in X]

In [ ]:
# Counts any special Character in a message
# Idea: maybe desperate or urgent messages contain more of those (e.g. exclamation marks etc.)
class SpecialCharacterCounter(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return [len(re.sub('[ a-zA-Z0-9]', '', x)) for x in X]

In [ ]:
def build_pipeline(model, mparams={}):
    '''
    model:   Class (!) of the Scikit-Learn model which should be used for classification
    mparams: Dict containing the parameters for the model
    '''
    pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()) 
            ])),
            ('msg_length', MessageLengthTransformer()),
            ('special_char', SpecialCharacterCounter()),
        ])),
        ('scaler', StandardScaler(with_mean=False)),
        ('clf', MultiOutputClassifier(model(**mparams)))
    ])
    return pipeline

In [ ]:
from sklearn.svm import SVC

svc_pipeline = build_pipeline(SVC, mparams={'gamma' : 'scale', 'C': 2.0})

In [ ]:
svc_pipeline.fit(X_train[:10], y_train[:10])

### 9. Export your model as a pickle file

In [ ]:
def save_model(model, folder='models/', filename=''):
    '''
    Saves a Modle in a folder.
    Args:
    - model:               object which gets saved
    - folder (optional):   folderpath of the file to be saved
    - filename (optional): name of the file to be saved
    '''
    # Input Handling
    if filename == '':
        filename = model.__class__.__name__
    if folder.endswith('/'):
        path = folder + filename
    else:
        path = folder + '/' + filename
        
    # Check if given folderpath exists
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    # Save the Model in the given folder
    with open(path, 'wb') as file:
        pickle.dump(model, file)
    
def load_model(path):
    '''
    Load a model given a path.
    Args:
    - path: String containing the path to the file
    Return:
    - model
    '''
    with open(path, "rb") as file:
        return pickle.load(file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.